#Tarea 2
#Universidad Autónoma de Nuevo León
#Facultad de Ciencias Físico-Matemáticas
##Maestría en Ciencia de Datos
###Datos Masivos
####Miguel Alejandro Andres Bojórquez
####1837555


In [1]:

!sudo apt update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
#Check this site for the latest download link https://www.apache.org/dyn/closer.lua/spark/spark-3.2.1/spark-3.2.1-bin-hadoop3.2.tgz
!wget -q https://archive.apache.org/dist/spark/spark-3.2.1/spark-3.2.1-bin-hadoop3.2.tgz
!tar xf spark-3.2.1-bin-hadoop3.2.tgz
!pip install -q findspark
!pip install pyspark
!pip install py4j
!sudo apt update



Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Ign:5 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Get:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:7 https://r2u.stat.illinois.edu/ubuntu jammy Release [5,713 B]
Get:8 https://r2u.stat.illinois.edu/ubuntu jammy Release.gpg [793 B]
Get:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [1,103 kB]
Get:10 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Hit:11 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:12 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:13 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
G

In [25]:

import os
import sys
import findspark
findspark.init()
findspark.find()

from pyspark import SparkContext

import pandas as pd
from google.colab import drive
import os
import sys

import findspark
findspark.init()
findspark.find()

import pyspark

from pyspark.sql import DataFrame, SparkSession
from typing import List
import pyspark.sql.types as T
import pyspark.sql.functions as F
import time

from pyspark.ml.feature import StringIndexer, VectorAssembler, StandardScaler
from pyspark.ml.classification import LogisticRegression, DecisionTreeClassifier, RandomForestClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

# **Explicación de base de datos**
El conjunto de datos que he elegido para mi tarea es un extenso registro de accidentes automovilísticos que abarca 49 estados de EE.UU. Este dataset se ha recopilado entre febrero de 2016 y marzo de 2023, utilizando múltiples APIs que proporcionan datos en tiempo real sobre incidentes de tráfico. Estas APIs obtienen información de diversas entidades, incluyendo departamentos de transporte, agencias de aplicación de la ley, cámaras de tráfico y sensores en las redes viales.

Este conjunto de datos es particularmente valioso ya que contiene aproximadamente 7.7 millones de registros de accidentes, lo que lo convierte en una base de datos muy grande para analizar. La riqueza en la cantidad de registros y la diversidad de columnas permiten realizar un análisis detallado sobre las tendencias y factores asociados a los accidentes.

He elegido este conjunto de datos porque es la base de datos más grande que encontré en los repositorios que revisé, y ofrece información tratada y relevante para llevar a cabo operaciones analíticas. Su amplitud tanto en registros como en columnas facilita el desarrollo de modelos predictivos, la identificación de patrones y la evaluación de políticas de seguridad vial. Esto lo convierte en un recurso ideal para realizar un análisis profundo y significativo sobre la accidentalidad en las carreteras.

https://www.kaggle.com/datasets/sobhanmoosavi/us-accidents

In [14]:

drive.mount('/content/drive')
spark = SparkSession.builder.appName("US_Accidentes").master("local[*]").getOrCreate()

file_path = "/content/drive/MyDrive/Datos Masivos/Datos/US_Accidents_March23.csv"
df = spark.read.csv(file_path, header=True, inferSchema=True)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
df.show()

+----+-------+--------+-------------------+-------------------+------------------+------------------+-------+-------+------------+--------------------+--------------------+------------+----------+-----+----------+-------+----------+------------+-------------------+--------------+-------------+-----------+------------+--------------+--------------+---------------+-----------------+-----------------+-------+-----+--------+--------+--------+-------+-------+----------+-------+-----+---------------+--------------+------------+--------------+--------------+-----------------+---------------------+
|  ID| Source|Severity|         Start_Time|           End_Time|         Start_Lat|         Start_Lng|End_Lat|End_Lng|Distance(mi)|         Description|              Street|        City|    County|State|   Zipcode|Country|  Timezone|Airport_Code|  Weather_Timestamp|Temperature(F)|Wind_Chill(F)|Humidity(%)|Pressure(in)|Visibility(mi)|Wind_Direction|Wind_Speed(mph)|Precipitation(in)|Weather_Condition|A

In [ ]:
df.printSchema()

root
 |-- ID: string (nullable = true)
 |-- Source: string (nullable = true)
 |-- Severity: integer (nullable = true)
 |-- Start_Time: timestamp (nullable = true)
 |-- End_Time: timestamp (nullable = true)
 |-- Start_Lat: double (nullable = true)
 |-- Start_Lng: double (nullable = true)
 |-- End_Lat: double (nullable = true)
 |-- End_Lng: double (nullable = true)
 |-- Distance(mi): double (nullable = true)
 |-- Description: string (nullable = true)
 |-- Street: string (nullable = true)
 |-- City: string (nullable = true)
 |-- County: string (nullable = true)
 |-- State: string (nullable = true)
 |-- Zipcode: string (nullable = true)
 |-- Country: string (nullable = true)
 |-- Timezone: string (nullable = true)
 |-- Airport_Code: string (nullable = true)
 |-- Weather_Timestamp: timestamp (nullable = true)
 |-- Temperature(F): double (nullable = true)
 |-- Wind_Chill(F): double (nullable = true)
 |-- Humidity(%): double (nullable = true)
 |-- Pressure(in): double (nullable = true)
 |-- V

In [15]:

df = df.withColumn(
    "Duration",
    (F.unix_timestamp(df['End_Time']) - F.unix_timestamp(df['Start_Time'])) / 60  # Diferencia en minutos
)

In [16]:
from pyspark.ml.feature import StringIndexer

indexer = StringIndexer(inputCol='Sunrise_Sunset', outputCol='Sunrise_Sunset_Index', handleInvalid="skip")
df = indexer.fit(df).transform(df)

In [17]:
columns_to_check = ["Duration", "Temperature(F)", "Wind_Chill(F)", "Humidity(%)", "Pressure(in)", "Visibility(mi)", "Severity"]
df_cleaned = df.dropna(subset=columns_to_check)

In [18]:
assembler = VectorAssembler(inputCols=["Duration", "Temperature(F)", "Wind_Chill(F)",
                                        "Humidity(%)", "Pressure(in)", "Visibility(mi)",
                                        "Sunrise_Sunset_Index"],
                            outputCol="features")
df_features = assembler.transform(df_cleaned)

In [10]:
df_features.show()

+----+-------+--------+-------------------+-------------------+------------------+------------------+-------+-------+------------+--------------------+--------------------+------------+----------+-----+----------+-------+----------+------------+-------------------+--------------+-------------+-----------+------------+--------------+--------------+---------------+-----------------+-----------------+-------+-----+--------+--------+--------+-------+-------+----------+-------+-----+---------------+--------------+------------+--------------+--------------+-----------------+---------------------+--------+--------------------+--------------------+
|  ID| Source|Severity|         Start_Time|           End_Time|         Start_Lat|         Start_Lng|End_Lat|End_Lng|Distance(mi)|         Description|              Street|        City|    County|State|   Zipcode|Country|  Timezone|Airport_Code|  Weather_Timestamp|Temperature(F)|Wind_Chill(F)|Humidity(%)|Pressure(in)|Visibility(mi)|Wind_Direction|Wi

In [19]:
scaler = StandardScaler(inputCol="features", outputCol="features_scaled")
scaler_model = scaler.fit(df_features)
df_scaled = scaler_model.transform(df_features)

In [20]:
train, test = df_scaled.randomSplit([0.7, 0.3], seed=42)

In [29]:
# Entrenamiento del modelo de bosque aleatorio
rf_classifier = RandomForestClassifier(featuresCol='features_scaled', labelCol='Severity', numTrees=10, maxDepth=5)
rf_model = rf_classifier.fit(train)

# Predicciones con el bosque aleatorio
rf_predictions = rf_model.transform(test)

In [22]:
dt = DecisionTreeClassifier(featuresCol='features_scaled', labelCol='Severity')
dt_model = dt.fit(train)

# Predicciones
dt_predictions = dt_model.transform(test)

In [30]:
evaluator = MulticlassClassificationEvaluator(labelCol='Severity', predictionCol='prediction', metricName='accuracy')

# Evaluación para el árbol de decisión
dt_accuracy = evaluator.evaluate(dt_predictions)
print(f"Accuracy of Decision Tree: {dt_accuracy}")

# Evaluación para el bosque aleatorio
rf_accuracy = evaluator.evaluate(rf_predictions)
print(f"Accuracy of Random Forest: {rf_accuracy}")

Accuracy of Decision Tree: 0.8410075667906662
Accuracy of Random Forest: 0.8410075667906662
